In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

In [7]:

# Load the dataset
df_clinical = pd.read_csv("C:/Users/heung/OneDrive/Documents/scData/cgga_mRNAseq_325/CGGA.mRNAseq_325_clinical.20200506.txt", sep="\t")
df_clinical.rename(columns = {'Censor (alive=0; dead=1)': 'Censor'}, inplace = True)

columns_to_keep = ['CGGA_ID', 'PRS_type', 'Grade', 'OS', 'Censor']
df_clinical = df_clinical[columns_to_keep]
df_clinical.tail(5)

CGGA_ID PRS_type    Grade      OS  Censor
320  CGGA_243  Primary   WHO II  2977.0     0.0
321  CGGA_247  Primary  WHO III   609.0     1.0
322  CGGA_738  Primary   WHO II  4132.0     0.0
323  CGGA_759  Primary   WHO IV  1263.0     1.0
324  CGGA_D30  Primary   WHO IV   215.0     1.0

In [8]:
df_clinical['PRS_type'].value_counts()

PRS_type
Primary      229
Recurrent     62
Secondary     30
Name: count, dtype: int64

In [11]:
df_temp = pd.read_excel('gain_geneset.xlsx')

gain_genes = df_temp.iloc[:, 0].tolist()

In [14]:
len(gain_genes)

1154

In [29]:
# Load the dataset
df_genes = pd.read_csv("C:/Users/heung/OneDrive/Documents/scData/cgga_mRNAseq_325/CGGA.mRNAseq_325.RSEM-genes.20200506.txt", sep="\t")

# Transpose the dataframe
df_genes = df_genes.T
df_genes.rename(columns = df_genes.iloc[0], inplace = True)
df_genes = df_genes.drop(df_genes.index[0])

df_genes.reset_index(inplace=True)
df_genes.rename(columns = {'index': 'CGGA_ID'}, inplace = True)

# Drop the columns that are not needed
# df1 = df.drop(columns = ['samples', 'cancer type abbreviation'], inplace = False)
# df1.head(5)

# Keep the columns that are needed
columns_to_keep = [col for col in gain_genes if col in df_genes.columns]

df_genes = df_genes[['CGGA_ID'] + columns_to_keep]


df_genes.tail(5)

CGGA_ID   AASS ABCA13  ABCB1 ABCB4 ABCB5  ABCB8  ABCF2 ABHD11   ACHE  \
320  CGGA_243   7.39   0.02   8.22  1.55   0.0   7.81  24.37  10.29   3.39   
321  CGGA_247  18.09   0.29   6.13  1.03  0.09  13.36  22.01  11.25   2.66   
322  CGGA_738  22.54   1.01  15.89  1.53  0.19  27.94  28.72  17.74  18.81   
323  CGGA_759  16.61   1.08  14.85  1.14   0.0  20.81  21.38   14.9  13.82   
324  CGGA_D30   5.57   0.15  14.73  1.89   0.0  14.32  21.45  18.22  12.18   

     ... ZNF862 ZNF890P  ZNF92  ZNHIT1 ZNRF2 ZNRF2P1 ZNRF2P2   ZP3 ZSCAN21  \
320  ...   7.75    0.11  14.08   24.01  3.48    0.32    2.47   0.0    7.53   
321  ...  48.01    0.85  14.51   29.08  3.64     0.0    0.91  0.74    9.18   
322  ...  15.13    0.32   6.47   55.57  5.67    0.69    3.64  0.45   10.07   
323  ...  12.43    0.25  11.28   84.91  7.98     0.0    3.13  0.43   10.12   
324  ...   7.77    0.27   5.89  114.33  8.57    0.48    2.25  3.33    7.42   

       ZYX  
320  21.45  
321  67.38  
322  72.42  
323  86.75  
324  85.42  

[5 rows x 804 columns]

In [30]:
merged_df = pd.merge(df_clinical, df_genes, on='CGGA_ID', how = 'inner')
merged_df = merged_df.dropna() # remove NaN
merged_df.tail(10)

CGGA_ID   PRS_type    Grade      OS  Censor   AASS ABCA13  ABCB1 ABCB4  \
315  CGGA_1246    Primary  WHO III  2972.0     0.0  12.16   0.06  11.71  0.34   
316  CGGA_1275    Primary   WHO IV   183.0     1.0   8.71   0.36   5.61  8.54   
317  CGGA_1450  Secondary   WHO IV   957.0     1.0   7.13   0.02   6.93  0.77   
318  CGGA_1460  Secondary   WHO IV   209.0     1.0  22.76   0.17   6.84  0.95   
319  CGGA_1475  Secondary   WHO IV   182.0     1.0  12.89   1.34   2.96  1.49   
320   CGGA_243    Primary   WHO II  2977.0     0.0   7.39   0.02   8.22  1.55   
321   CGGA_247    Primary  WHO III   609.0     1.0  18.09   0.29   6.13  1.03   
322   CGGA_738    Primary   WHO II  4132.0     0.0  22.54   1.01  15.89  1.53   
323   CGGA_759    Primary   WHO IV  1263.0     1.0  16.61   1.08  14.85  1.14   
324   CGGA_D30    Primary   WHO IV   215.0     1.0   5.57   0.15  14.73  1.89   

    ABCB5  ... ZNF862 ZNF890P  ZNF92  ZNHIT1 ZNRF2 ZNRF2P1 ZNRF2P2   ZP3  \
315  0.02  ...   9.32    0.31  12.31   41.66  2.67    0.74    1.11  3.19   
316  0.49  ...   7.72     0.0   1.47  106.53   4.8     0.6    0.33  1.15   
317   0.0  ...   11.5     0.0    4.2   79.41  2.05     1.1    4.52  0.24   
318   0.0  ...  13.77     0.5   4.65   89.46   2.8    0.52     2.7  0.37   
319  0.03  ...  10.23     0.0   8.89   69.36  5.17    5.92     6.2  0.06   
320   0.0  ...   7.75    0.11  14.08   24.01  3.48    0.32    2.47   0.0   
321  0.09  ...  48.01    0.85  14.51   29.08  3.64     0.0    0.91  0.74   
322  0.19  ...  15.13    0.32   6.47   55.57  5.67    0.69    3.64  0.45   
323   0.0  ...  12.43    0.25  11.28   84.91  7.98     0.0    3.13  0.43   
324   0.0  ...   7.77    0.27   5.89  114.33  8.57    0.48    2.25  3.33   

    ZSCAN21     ZYX  
315   10.21   26.78  
316     6.9  301.38  
317    8.58   60.35  
318   12.84   243.2  
319   12.03  112.77  
320    7.53   21.45  
321    9.18   67.38  
322   10.07   72.42  
323   10.12   86.75  
324    7.42   85.42  

[10 rows x 808 columns]

In [42]:
merged_df

CGGA_ID PRS_type    Grade      OS  Censor   AASS ABCA13  ABCB1 ABCB4  \
0    CGGA_1001  Primary   WHO IV  3817.0     0.0   6.89   0.68   1.82  1.29   
1    CGGA_1006  Primary  WHO III   254.0     1.0   5.84   1.27   14.0  4.52   
2    CGGA_1007  Primary   WHO IV   345.0     1.0  10.57   0.19  28.82  9.35   
3    CGGA_1011  Primary   WHO IV   109.0     1.0   11.4   0.78  16.15  6.25   
4    CGGA_1015  Primary   WHO IV   164.0     1.0   8.56   1.97   1.83  1.51   
..         ...      ...      ...     ...     ...    ...    ...    ...   ...   
320   CGGA_243  Primary   WHO II  2977.0     0.0   7.39   0.02   8.22  1.55   
321   CGGA_247  Primary  WHO III   609.0     1.0  18.09   0.29   6.13  1.03   
322   CGGA_738  Primary   WHO II  4132.0     0.0  22.54   1.01  15.89  1.53   
323   CGGA_759  Primary   WHO IV  1263.0     1.0  16.61   1.08  14.85  1.14   
324   CGGA_D30  Primary   WHO IV   215.0     1.0   5.57   0.15  14.73  1.89   

    ABCB5  ... ZNF862 ZNF890P  ZNF92  ZNHIT1  ZNRF2 ZNRF2P1 ZNRF2P2   ZP3  \
0    0.08  ...   9.81    0.23   4.72   50.52   13.2     0.0    1.82  1.75   
1    0.92  ...  15.06    0.33   7.73  150.96   4.14    0.94    2.54  9.17   
2    0.01  ...  16.46    0.22   5.23   59.32   5.58    3.01    5.95  2.04   
3    0.23  ...   8.19    0.45   4.21   39.79  10.62     0.0    1.06  0.66   
4    0.33  ...  11.24    0.33   7.31   72.69   7.89    0.26    0.61  1.94   
..    ...  ...    ...     ...    ...     ...    ...     ...     ...   ...   
320   0.0  ...   7.75    0.11  14.08   24.01   3.48    0.32    2.47   0.0   
321  0.09  ...  48.01    0.85  14.51   29.08   3.64     0.0    0.91  0.74   
322  0.19  ...  15.13    0.32   6.47   55.57   5.67    0.69    3.64  0.45   
323   0.0  ...  12.43    0.25  11.28   84.91   7.98     0.0    3.13  0.43   
324   0.0  ...   7.77    0.27   5.89  114.33   8.57    0.48    2.25  3.33   

    ZSCAN21     ZYX  
0      8.76  158.36  
1     11.94  761.56  
2      8.68   92.85  
3      6.64  158.75  
4      4.94  380.73  
..      ...     ...  
320    7.53   21.45  
321    9.18   67.38  
322   10.07   72.42  
323   10.12   86.75  
324    7.42   85.42  

[309 rows x 808 columns]

In [36]:
result_df = pd.DataFrame(index = columns_to_keep)
result_df['log_rank'] = np.nan
result_df['p_value'] = np.nan
result_df['survival'] = np.nan

# LOG RANK analysis

In [43]:
from lifelines.statistics import logrank_test

def survival_analysis(gene):
    # split the group
    quartiles = pd.qcut(merged_df[gene], q = 2, labels = ['low', 'high'])
    low_df = merged_df[quartiles == 'low']
    high_df = merged_df[quartiles == 'high']
    
    result = logrank_test(low_df['OS'],
                        high_df['OS'],
                        event_observed_A = low_df['Censor'],
                        event_observed_B = high_df['Censor'])

    median_high = high_df['OS'].median()
    median_low = low_df['OS'].median()

    result_df.at[gene, 'log_rank'] = result.test_statistic
    result_df.at[gene, 'p_value'] = result.p_value
    
    if median_high > median_low:
        result_df.at[gene, 'survival'] = 'high'
    elif median_high < median_low:
        result_df.at[gene, 'survival'] = 'low'
    else:
        result_df.at[gene, 'survival'] = 'same'
    


In [45]:
for gene in columns_to_keep:
    survival_analysis(gene)

In [46]:
result_df

log_rank       p_value survival
AASS       2.269337  1.319564e-01      low
ABCA13    29.136823  6.744289e-08      low
ABCB1     12.291567  4.550095e-04     high
ABCB4     41.539296  1.155253e-10      low
ABCB5      7.705491  5.505310e-03      low
...             ...           ...      ...
ZNRF2P1    2.168065  1.409035e-01     high
ZNRF2P2    7.891946  4.965540e-03     high
ZP3       12.113750  5.005137e-04      low
ZSCAN21   10.730151  1.054038e-03      low
ZYX      103.947021  2.077988e-24      low

[803 rows x 3 columns]

In [56]:
filtered_result_df = result_df[result_df['p_value'] < 0.001]
filtered_result_df

log_rank       p_value survival
ABCA13   29.136823  6.744289e-08      low
ABCB1    12.291567  4.550095e-04     high
ABCB4    41.539296  1.155253e-10      low
ABCF2    25.010191  5.702810e-07      low
ACN9     31.892828  1.629176e-08      low
...            ...           ...      ...
ZNF853   11.268696  7.882507e-04     high
ZNHIT1   49.363542  2.126613e-12      low
ZNRF2    55.089260  1.151785e-13      low
ZP3      12.113750  5.005137e-04      low
ZYX     103.947021  2.077988e-24      low

[439 rows x 3 columns]

In [57]:
filtered_result_df.to_csv('gain_geneset_survival_analysis.csv')